In [1]:
import pandas as pd
import numpy as np
import re # Regular expression library

# StringIO - to read dataframe from string
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [2]:
cd data

C:\Users\silveto\Documents\PROJECTS\FL\Jupyter\data


In [3]:
# Read all tables
df = pd.read_csv("df.csv")
df_a = pd.read_csv("df-a.csv")
df_aggr_status = pd.read_csv("df-agg-status.csv")
df_summary = pd.read_csv("Summary.csv")

print(df.shape, df_a.shape, df_aggr_status.shape, df_summary.shape)

(630, 10) (16, 9) (315, 6) (270010, 31)


In [4]:
# Convert date strings to datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
df_a['date'] = pd.to_datetime(df_a['date'], format='%Y-%m-%d %H:%M:%S')
df_aggr_status['date'] = pd.to_datetime(df_aggr_status['date'], format='%Y-%m-%d %H:%M:%S')
df_summary['Timestamp'] = pd.to_datetime(df_summary['Timestamp'], format='%d/%m/%Y %H:%M:%S')

In [5]:
# Remove .snapshot values (?)
df = df[~df['Filesystem'].str.contains('.snapshot')]
df_a = df_a[~df_a['Aggregate'].str.contains('.snapshot')]

In [6]:
# Get name of the volume from Filesystem
df['vol'] = df['Filesystem']
df['vol'] = df['vol'].str.replace('/vol/','').replace('/','')
df['vol'] = df['vol'].str.replace('/','')

In [7]:
# Merge tables
df_merged = pd.merge(left=df,
               right=df_aggr_status[['date','agg','vol']],
               how='left',
               left_on=['date','vol'],
               right_on=['date','vol'])
df_merged = pd.merge(left=df_merged,
               right=df_a[['date','Aggregate','kbytes','used', 'avail', 'capacity']],
               how='left',
               left_on=['date','agg'],
               right_on=['date','Aggregate'],
               suffixes=('_vol', '_agg'))
df_merged.shape

(315, 17)

In [8]:
df_merged.head()

,date,ASUP_Type,hostname,serial_no,Filesystem,kbytes_vol,used_vol,avail_vol,capacity_vol,Mounted on,vol,agg,Aggregate,kbytes_agg,used_agg,avail_agg,capacity_agg
0,2017-07-02 00:34:30,WEEKLY_LOG,NASKNX07A,451416000026,/vol/vol0/,274596860,9436936,265159924,3%,/vol/vol0/,vol0,aggr0,aggr0,326979552,290796292,36183260,89%
1,2017-07-02 00:34:30,WEEKLY_LOG,NASKNX07A,451416000026,/vol/vf_NAS_AURDIVIC_rootvol/,1992296,13404,1978892,1%,/vol/vf_NAS_AURDIVIC_rootvol/,vf_NAS_AURDIVIC_rootvol,aggr01_nsas,aggr01_nsas,115782104680,98382566932,17399537748,85%
2,2017-07-02 00:34:30,WEEKLY_LOG,NASKNX07A,451416000026,/vol/wd_aurdivic_hobo_vol01/,2040109468,2405664,2037703804,0%,/vol/wd_aurdivic_hobo_vol01/,wd_aurdivic_hobo_vol01,aggr01_nsas,aggr01_nsas,115782104680,98382566932,17399537748,85%
3,2017-07-02 00:34:30,WEEKLY_LOG,NASKNX07A,451416000026,/vol/wd_aurdivic_branch_vol01/,2040109468,110912,2039998556,0%,/vol/wd_aurdivic_branch_vol01/,wd_aurdivic_branch_vol01,aggr01_nsas,aggr01_nsas,115782104680,98382566932,17399537748,85%
4,2017-07-02 00:34:30,WEEKLY_LOG,NASKNX07A,451416000026,/vol/vf_NAS_AURDI02_rootvol/,1992296,15640,1976656,1%,/vol/vf_NAS_AURDI02_rootvol/,vf_NAS_AURDI02_rootvol,aggr01_nsas,aggr01_nsas,115782104680,98382566932,17399537748,85%
